## Additional analysis

We investigate how exposed content creators react to this new policy; more precisely, we examine the causal effect of the demonetisation policy on the nature of published content. 

In [1]:
import pandas as pd
import numpy as np
import re

## Channel dataset

In [2]:
db_chan=pd.read_csv('true_weekly_uploads_umbalanced_data.csv', sep=';')

db_chan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3267117 entries, 0 to 3267116
Data columns (total 15 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   Unnamed: 0.1             int64  
 1   Unnamed: 0               int64  
 2   channel                  object 
 3   week                     object 
 4   first_moderation_update  int64  
 5   id_0                     int64  
 6   type                     object 
 7   main_cat                 object 
 8   category                 object 
 9   datetime                 object 
 10  subs_x                   float64
 11  treated                  int64  
 12  date_formated            object 
 13  subs_y                   float64
 14  sub_bin_prior            object 
dtypes: float64(2), int64(5), object(8)
memory usage: 373.9+ MB


In [3]:
#Keeping same channels as for the econometric regression
db_chan=db_chan[db_chan['main_cat']!='Film & Animation']
db_chan=db_chan[db_chan['main_cat']!='Music']
db_chan=db_chan[db_chan['main_cat']!='News & Politics']
db_chan=db_chan[db_chan['main_cat']!='Sports']
db_chan=db_chan[db_chan['type']!='missing']
db_chan=db_chan[db_chan['type']!='control']
db_chan=db_chan[~db_chan['sub_bin_prior'].isna()]


db_chan['channel'].nunique()


20323

In [4]:
list_chan=db_chan['channel'].unique()
list_chan

array(['UC--70ql_IxJmhmqXqrkJrWQ', 'UC--LAVm36WiCmqQbekV17qQ',
       'UC--MxpGXJ3LVD8KvlNzRlcA', ..., 'UCzzYnZ8GIzfB1Vr3hk2Nj9Q',
       'UCzzzPMh9fZHxc7gxPRB2LHQ', 'UCzzzZ3-icktxbC3j7hkWqRw'],
      dtype=object)

In [5]:
del db_chan

## Video dataset

In [6]:
db_video=pd.read_csv('video_database_per_period_selected.csv', sep=';')
db_video.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11845978 entries, 0 to 11845977
Data columns (total 13 columns):
 #   Column          Dtype  
---  ------          -----  
 0   Unnamed: 0.1    int64  
 1   Unnamed: 0      int64  
 2   category        object 
 3   channel         object 
 4   date_crawled    object 
 5   description     object 
 6   id              object 
 7   duration        int64  
 8   tags            object 
 9   title           object 
 10  upload          object 
 11  week            object 
 12  before_updates  float64
dtypes: float64(1), int64(3), object(9)
memory usage: 1.1+ GB


In [7]:
db_video = db_video[db_video['channel'].isin(list_chan)]
db_video

,Unnamed: 0.1,Unnamed: 0,category,channel,date_crawled,description,id,duration,tags,title,upload,week,before_updates
141,270,536,Gaming,UCzWfqsjlvzAOUozsGFR3kbg,2019-10-31 15:41:58.124418,Clash royale first magical chest quest in clas...,tmW-gHCMixA,827,"clash royale,clash royal,clash royale quest,cl...",OPENING FIRST MAGICAL CHEST QUEST IN CLASH ROY...,2017-10-29,2017-43,0.0
142,271,537,Gaming,UCzWfqsjlvzAOUozsGFR3kbg,2019-10-31 15:41:58.844202,"Clash royale legendary, magical and giant ches...",IrQmluA7ol0,1140,"clash royale,clash royal,clash royale elite ch...","LEGENDARY, MAGICAL AND GIANT CHEST OPENING :: ...",2017-10-25,2017-43,0.0
143,272,538,Gaming,UCzWfqsjlvzAOUozsGFR3kbg,2019-10-31 15:41:59.602560,Clash royale kings cup challenge with tons of ...,Rw4OLBbm0lk,842,"clash royale,clash royal,clash royale kings cu...","KINGS CUP CHALLENGE :: Clash Royale :: GIANT, ...",2017-10-24,2017-43,0.0
144,273,539,Gaming,UCzWfqsjlvzAOUozsGFR3kbg,2019-10-31 15:42:00.225311,Download Castle Crush here: http://bit.ly/Cast...,MefLP5txBo0,828,"Castle Crush,castle crush legendary,castle cru...",NEW LEGENDARY DRAGON FROM CHEST OPENING :: Cas...,2017-10-23,2017-43,0.0
145,274,540,Gaming,UCzWfqsjlvzAOUozsGFR3kbg,2019-10-31 15:42:00.874368,Clash royale the next chest quest. Will this f...,UBnsame1Lss,847,"clash royale,clash royal,clash royale quest,cl...","OH PLEASE, SUPER MAGICAL CHEST QUEST :: Clash ...",2017-10-22,2017-42,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11844315,18696510,2367765,People & Blogs,UCs0hGQ5Wie-a7j8aIYAxxjQ,2019-11-13 21:44:02.197085,▽Products Mentioned:\n\nMorphe 35w Palette\nMo...,9QriMo2pyUM,413,"tutorial,makeup,beauty",△ Date Night Fall Edition ▽,2016-10-04,2016-40,1.0
11844316,18696511,2367766,People & Blogs,UCs0hGQ5Wie-a7j8aIYAxxjQ,2019-11-13 21:44:02.769199,▽Products Mentioned:\nStar Dust Palette Too Fa...,zbLmR6mMzEA,462,"makeup,toofaced,tutorial",△ Grunge Princess Leia ▽,2016-09-18,2016-37,1.0
11844317,18696512,2367767,People & Blogs,UCs0hGQ5Wie-a7j8aIYAxxjQ,2019-11-13 21:44:03.294655,▽Products mentioned:\nToo Faced Lip Injection ...,kTSQHcCID9c,48,"ipsy,glam,bag,makeup",△ September Ipsy Glam Bag ▽,2016-09-17,2016-37,1.0
11844318,18696513,2367768,People & Blogs,UCs0hGQ5Wie-a7j8aIYAxxjQ,2019-11-13 21:44:03.884569,▽Products Mentioned:\nArdell Wispie Lashes 600...,L2KWfSKw4ks,201,NaN,△ Quick Stacked Lashes Tutorial ▽,2016-09-14,2016-37,1.0


In [8]:
db_video['channel'].nunique()

20323

In [9]:
db_video['week'].max()

'2017-43'

### Keyword identification

In [10]:
pattern_sex=[' ass ', ' sex ', ' dick ', ' vagina ', ' penis ', ' anal ', ' anus ', ' erotic ', 
             ' boobs ', ' butt ', ' cum ', ' nudity ', ' sexual ', ' porn ', ' xxx ', ' fuck ', ' condom ']
 
pattern_sex = re.compile('|'.join(pattern_sex))   
mask = ((db_video['title'].str.contains(pattern_sex))|(db_video['description'].str.contains(pattern_sex))|(db_video['tags'].str.contains(pattern_sex)))
db_video['sex_content']=mask
db_video['sex_content']=db_video['sex_content'].fillna('None')

In [11]:
pattern_violence=[' blood ', ' torture ', ' murder ', ' abuse ', ' rape ', ' kill ', ' dead ', ' mass shooting ', 
                  ' execution ', ' kidnapping ', ' slaughter ', ' suicide ', ' victim ', 
                  ' violence ', ' violent ', ' weapon ', ' warefare ', ' gun', ' bomb', ' terrorism ', " extremism "]

pattern_violence= re.compile('|'.join(pattern_violence))   
mask_violence = ((db_video['title'].str.contains(pattern_violence))|(db_video['description'].str.contains(pattern_violence))|(db_video['tags'].str.contains(pattern_violence)))
db_video['violent_content']=mask_violence
db_video['violent_content']=db_video['violent_content'].fillna('No tags')

In [12]:
pattern_insults=[' bastard ', ' pussy ', ' dumbass ', ' goddamn ', ' bitch ', ' nigger ', ' shit ', ' idiot ', 
                 ' dammit ', ' cunt ']

pattern_insults= re.compile('|'.join(pattern_insults))   
mask_insults = ((db_video['title'].str.contains(pattern_insults))|(db_video['description'].str.contains(pattern_insults))|(db_video['tags'].str.contains(pattern_insults)))
db_video['insults']=mask_insults
db_video['insults']=db_video['insults'].fillna('No tags')

In [13]:
pattern_drug=[" acid ", ' drug ', ' weed ', ' boong ', ' cannabis ', ' cbd ', ' cocain ', ' crack ',
              ' dealer ', ' joint ', ' junky ', ' lsd ', ' marijuana ', ' rehab ', 
               ' stoned ',  ' thc ', ' heroin']

pattern_drug= re.compile('|'.join(pattern_drug))   
mask_drug = ((db_video['title'].str.contains(pattern_drug))|(db_video['description'].str.contains(pattern_drug))|(db_video['tags'].str.contains(pattern_drug)))
db_video['drug']=mask_drug
db_video['drug']=db_video['drug'].fillna('No tags')

In [14]:
pattern_sensitive=[" war ", " abortion ", " accident ", " Hitler ",  " fascism ",  " AIDS ", " Al Qaeda ", " alt right ", 
                   " genocide ", ' assassination ',
                   ' attack ', " concentration camp ",  " incel ",  " holocaust ",  " homophobia ", " illegal ", " incest ", " ISIS ", 
                   " Israel ", " Palestine ", " jewish ",  " Ku Klux Klan ", " LGBT ",   " nazi ", 
                   ' racism ', ' slavery ', ' supremacist ', ' supremacy ', ' transphobia ',  ' nuclear weapon ', ' climate change ', ' 9/11 ', ' Twin Towers ']

pattern_sensitive= re.compile('|'.join(pattern_sensitive))   
mask_sensitive = ((db_video['title'].str.contains(pattern_sensitive))|(db_video['description'].str.contains(pattern_sensitive))|(db_video['tags'].str.contains(pattern_sensitive)))
db_video['sensitive_content']=mask_sensitive
db_video['sensitive_content']=db_video['sensitive_content'].fillna('No tags')

In [15]:
db_video

,Unnamed: 0.1,Unnamed: 0,category,channel,date_crawled,description,id,duration,tags,title,upload,week,before_updates,sex_content,violent_content,insults,drug,sensitive_content
141,270,536,Gaming,UCzWfqsjlvzAOUozsGFR3kbg,2019-10-31 15:41:58.124418,Clash royale first magical chest quest in clas...,tmW-gHCMixA,827,"clash royale,clash royal,clash royale quest,cl...",OPENING FIRST MAGICAL CHEST QUEST IN CLASH ROY...,2017-10-29,2017-43,0.0,False,False,False,False,False
142,271,537,Gaming,UCzWfqsjlvzAOUozsGFR3kbg,2019-10-31 15:41:58.844202,"Clash royale legendary, magical and giant ches...",IrQmluA7ol0,1140,"clash royale,clash royal,clash royale elite ch...","LEGENDARY, MAGICAL AND GIANT CHEST OPENING :: ...",2017-10-25,2017-43,0.0,False,False,False,False,False
143,272,538,Gaming,UCzWfqsjlvzAOUozsGFR3kbg,2019-10-31 15:41:59.602560,Clash royale kings cup challenge with tons of ...,Rw4OLBbm0lk,842,"clash royale,clash royal,clash royale kings cu...","KINGS CUP CHALLENGE :: Clash Royale :: GIANT, ...",2017-10-24,2017-43,0.0,False,False,False,False,False
144,273,539,Gaming,UCzWfqsjlvzAOUozsGFR3kbg,2019-10-31 15:42:00.225311,Download Castle Crush here: http://bit.ly/Cast...,MefLP5txBo0,828,"Castle Crush,castle crush legendary,castle cru...",NEW LEGENDARY DRAGON FROM CHEST OPENING :: Cas...,2017-10-23,2017-43,0.0,False,True,False,False,False
145,274,540,Gaming,UCzWfqsjlvzAOUozsGFR3kbg,2019-10-31 15:42:00.874368,Clash royale the next chest quest. Will this f...,UBnsame1Lss,847,"clash royale,clash royal,clash royale quest,cl...","OH PLEASE, SUPER MAGICAL CHEST QUEST :: Clash ...",2017-10-22,2017-42,0.0,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11844315,18696510,2367765,People & Blogs,UCs0hGQ5Wie-a7j8aIYAxxjQ,2019-11-13 21:44:02.197085,▽Products Mentioned:\n\nMorphe 35w Palette\nMo...,9QriMo2pyUM,413,"tutorial,makeup,beauty",△ Date Night Fall Edition ▽,2016-10-04,2016-40,1.0,False,False,False,False,False
11844316,18696511,2367766,People & Blogs,UCs0hGQ5Wie-a7j8aIYAxxjQ,2019-11-13 21:44:02.769199,▽Products Mentioned:\nStar Dust Palette Too Fa...,zbLmR6mMzEA,462,"makeup,toofaced,tutorial",△ Grunge Princess Leia ▽,2016-09-18,2016-37,1.0,False,False,False,False,False
11844317,18696512,2367767,People & Blogs,UCs0hGQ5Wie-a7j8aIYAxxjQ,2019-11-13 21:44:03.294655,▽Products mentioned:\nToo Faced Lip Injection ...,kTSQHcCID9c,48,"ipsy,glam,bag,makeup",△ September Ipsy Glam Bag ▽,2016-09-17,2016-37,1.0,False,False,False,False,False
11844318,18696513,2367768,People & Blogs,UCs0hGQ5Wie-a7j8aIYAxxjQ,2019-11-13 21:44:03.884569,▽Products Mentioned:\nArdell Wispie Lashes 600...,L2KWfSKw4ks,201,NaN,△ Quick Stacked Lashes Tutorial ▽,2016-09-14,2016-37,1.0,False,False,False,False,False


In [16]:
condition=[(db_video['sex_content']==True) | (db_video['violent_content']==True) | (db_video['insults']==True) | (db_video['drug']==True) | (db_video['sensitive_content']==True)]

value=[1]

db_video['naf_content']=np.select(condition, value)
db_video

,Unnamed: 0.1,Unnamed: 0,category,channel,date_crawled,description,id,duration,tags,title,upload,week,before_updates,sex_content,violent_content,insults,drug,sensitive_content,naf_content
141,270,536,Gaming,UCzWfqsjlvzAOUozsGFR3kbg,2019-10-31 15:41:58.124418,Clash royale first magical chest quest in clas...,tmW-gHCMixA,827,"clash royale,clash royal,clash royale quest,cl...",OPENING FIRST MAGICAL CHEST QUEST IN CLASH ROY...,2017-10-29,2017-43,0.0,False,False,False,False,False,0
142,271,537,Gaming,UCzWfqsjlvzAOUozsGFR3kbg,2019-10-31 15:41:58.844202,"Clash royale legendary, magical and giant ches...",IrQmluA7ol0,1140,"clash royale,clash royal,clash royale elite ch...","LEGENDARY, MAGICAL AND GIANT CHEST OPENING :: ...",2017-10-25,2017-43,0.0,False,False,False,False,False,0
143,272,538,Gaming,UCzWfqsjlvzAOUozsGFR3kbg,2019-10-31 15:41:59.602560,Clash royale kings cup challenge with tons of ...,Rw4OLBbm0lk,842,"clash royale,clash royal,clash royale kings cu...","KINGS CUP CHALLENGE :: Clash Royale :: GIANT, ...",2017-10-24,2017-43,0.0,False,False,False,False,False,0
144,273,539,Gaming,UCzWfqsjlvzAOUozsGFR3kbg,2019-10-31 15:42:00.225311,Download Castle Crush here: http://bit.ly/Cast...,MefLP5txBo0,828,"Castle Crush,castle crush legendary,castle cru...",NEW LEGENDARY DRAGON FROM CHEST OPENING :: Cas...,2017-10-23,2017-43,0.0,False,True,False,False,False,1
145,274,540,Gaming,UCzWfqsjlvzAOUozsGFR3kbg,2019-10-31 15:42:00.874368,Clash royale the next chest quest. Will this f...,UBnsame1Lss,847,"clash royale,clash royal,clash royale quest,cl...","OH PLEASE, SUPER MAGICAL CHEST QUEST :: Clash ...",2017-10-22,2017-42,0.0,False,False,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11844315,18696510,2367765,People & Blogs,UCs0hGQ5Wie-a7j8aIYAxxjQ,2019-11-13 21:44:02.197085,▽Products Mentioned:\n\nMorphe 35w Palette\nMo...,9QriMo2pyUM,413,"tutorial,makeup,beauty",△ Date Night Fall Edition ▽,2016-10-04,2016-40,1.0,False,False,False,False,False,0
11844316,18696511,2367766,People & Blogs,UCs0hGQ5Wie-a7j8aIYAxxjQ,2019-11-13 21:44:02.769199,▽Products Mentioned:\nStar Dust Palette Too Fa...,zbLmR6mMzEA,462,"makeup,toofaced,tutorial",△ Grunge Princess Leia ▽,2016-09-18,2016-37,1.0,False,False,False,False,False,0
11844317,18696512,2367767,People & Blogs,UCs0hGQ5Wie-a7j8aIYAxxjQ,2019-11-13 21:44:03.294655,▽Products mentioned:\nToo Faced Lip Injection ...,kTSQHcCID9c,48,"ipsy,glam,bag,makeup",△ September Ipsy Glam Bag ▽,2016-09-17,2016-37,1.0,False,False,False,False,False,0
11844318,18696513,2367768,People & Blogs,UCs0hGQ5Wie-a7j8aIYAxxjQ,2019-11-13 21:44:03.884569,▽Products Mentioned:\nArdell Wispie Lashes 600...,L2KWfSKw4ks,201,NaN,△ Quick Stacked Lashes Tutorial ▽,2016-09-14,2016-37,1.0,False,False,False,False,False,0


In [17]:
db_video=db_video.replace([False, True], 
                         [0,1])

In [18]:
db_id=db_video.groupby(['week', 'channel'], 
                           as_index=False)['id'].count()
db_id

,week,channel,id
0,2016-33,UC--LAVm36WiCmqQbekV17qQ,1
1,2016-33,UC--MxpGXJ3LVD8KvlNzRlcA,1
2,2016-33,UC--j-yuiKFQhNAUYLt3v4fA,3
3,2016-33,UC--szL7ECbL2jmnOpx6BrNg,1
4,2016-33,UC-0CzRZeML8zw4pFTVDq65Q,1
...,...,...,...
882607,2017-43,UCzzOKT1SkE7qdycJvPCR3Gg,1
882608,2017-43,UCzzQnR2e5VIgziK3SiJ6uIg,2
882609,2017-43,UCzzSPRyMYUw6IGueHr1K0Jg,7
882610,2017-43,UCzzYnZ8GIzfB1Vr3hk2Nj9Q,6


In [19]:
db_naf=db_video.groupby(['week', 'channel'], as_index=False)['naf_content'].sum()
db_naf

,week,channel,naf_content
0,2016-33,UC--LAVm36WiCmqQbekV17qQ,0
1,2016-33,UC--MxpGXJ3LVD8KvlNzRlcA,0
2,2016-33,UC--j-yuiKFQhNAUYLt3v4fA,0
3,2016-33,UC--szL7ECbL2jmnOpx6BrNg,0
4,2016-33,UC-0CzRZeML8zw4pFTVDq65Q,0
...,...,...,...
882607,2017-43,UCzzOKT1SkE7qdycJvPCR3Gg,0
882608,2017-43,UCzzQnR2e5VIgziK3SiJ6uIg,0
882609,2017-43,UCzzSPRyMYUw6IGueHr1K0Jg,0
882610,2017-43,UCzzYnZ8GIzfB1Vr3hk2Nj9Q,0


In [20]:
db_grouped=pd.merge(db_id, db_naf, how='left', left_on=['week', 'channel'], right_on= ['week', 'channel'])
db_grouped

,week,channel,id,naf_content
0,2016-33,UC--LAVm36WiCmqQbekV17qQ,1,0
1,2016-33,UC--MxpGXJ3LVD8KvlNzRlcA,1,0
2,2016-33,UC--j-yuiKFQhNAUYLt3v4fA,3,0
3,2016-33,UC--szL7ECbL2jmnOpx6BrNg,1,0
4,2016-33,UC-0CzRZeML8zw4pFTVDq65Q,1,0
...,...,...,...,...
882607,2017-43,UCzzOKT1SkE7qdycJvPCR3Gg,1,0
882608,2017-43,UCzzQnR2e5VIgziK3SiJ6uIg,2,0
882609,2017-43,UCzzSPRyMYUw6IGueHr1K0Jg,7,0
882610,2017-43,UCzzYnZ8GIzfB1Vr3hk2Nj9Q,6,0


In [21]:
db_grouped['week']=db_grouped['week'].replace('2017-00', '2016-52')

In [22]:
db_grouped_test=db_grouped.groupby(['week', 'channel'], 
                                   as_index=False)['id', 'naf_content'].sum()

db_grouped_test

C:\Users\mrass\AppData\Local\Temp\ipykernel_23316\1406123930.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  db_grouped_test=db_grouped.groupby(['week', 'channel'],


,week,channel,id,naf_content
0,2016-33,UC--LAVm36WiCmqQbekV17qQ,1,0
1,2016-33,UC--MxpGXJ3LVD8KvlNzRlcA,1,0
2,2016-33,UC--j-yuiKFQhNAUYLt3v4fA,3,0
3,2016-33,UC--szL7ECbL2jmnOpx6BrNg,1,0
4,2016-33,UC-0CzRZeML8zw4pFTVDq65Q,1,0
...,...,...,...,...
878801,2017-43,UCzzOKT1SkE7qdycJvPCR3Gg,1,0
878802,2017-43,UCzzQnR2e5VIgziK3SiJ6uIg,2,0
878803,2017-43,UCzzSPRyMYUw6IGueHr1K0Jg,7,0
878804,2017-43,UCzzYnZ8GIzfB1Vr3hk2Nj9Q,6,0


In [23]:
db_grouped_test[db_grouped_test['week']=='2016-52']

,week,channel,id,naf_content
267086,2016-52,UC--70ql_IxJmhmqXqrkJrWQ,1,0
267087,2016-52,UC--j-yuiKFQhNAUYLt3v4fA,7,0
267088,2016-52,UC-0CzRZeML8zw4pFTVDq65Q,2,0
267089,2016-52,UC-0Wg1wOUp-bRF1ZiC0dE8g,6,1
267090,2016-52,UC-1CvlOVGAUzvffUyLOQ0hA,1,0
...,...,...,...,...
280273,2016-52,UCzyuszXJUbY6w-S_t3n7hNQ,2,0
280274,2016-52,UCzzQnR2e5VIgziK3SiJ6uIg,6,0
280275,2016-52,UCzzSPRyMYUw6IGueHr1K0Jg,4,0
280276,2016-52,UCzzYnZ8GIzfB1Vr3hk2Nj9Q,7,0


In [24]:
db_grouped_test.to_csv('week_keywords.csv', sep=';')